### Data
* Combining many datsets and balancing the data for model training

### Library imports

In [127]:
import json
import os
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix

In [106]:
!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable


In [128]:
Output_Dir = "./final_data"
os.makedirs(Output_Dir, exist_ok=True)

* All excel files combined

In [40]:
## combine all files to single file for trainging model
folder_path = "./data_biasingmodel"
all_data = []

# Loop through all .xlsx files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_excel(file_path)
            all_data.append(df)
        except Exception as e:
            print(f"Failed to read {filename}: {e}")

# Combine all the DataFrames
combined_df = pd.concat(all_data, ignore_index=True)

# Save to a single Excel or CSV file
combined_df.to_excel(os.path.join(folder_path, "combined_data.xlsx"), index=False)
print(f"Combined {len(all_data)} files with total {len(combined_df)} rows.")

/Users/pranavi/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


KeyboardInterrupt: 

In [75]:
!pip install missingno

Defaulting to user installation because normal site-packages is not writeable
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 10.9 MB/s eta 0:00:00a 0:00:01
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.0 MB/s eta 0:00:00a 0:00:01


In [129]:
df_xl = pd.read_excel("./data_biasingmodel/combined_data.xlsx", engine='openpyxl')

In [130]:
df_xl.head()

,text,news_link,outlet,topic,type,group_id,num_sent,label_bias,label_opinion,article,biased_words,Label_bias_0-1,annotator_id,df_id
0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1.0,1.0,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"['belated', 'birtherism']",NaN,NaN,NaN
1,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,1.0,1.0,Biased,Expresses writer’s opinion,Speaking to the country for the first time fro...,['crisis'],NaN,NaN,NaN
2,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,1.0,1.0,Biased,Somewhat factual but also opinionated,The left has a thing for taking babies hostage...,"['killing', 'never', 'developing', 'humans', '...",NaN,NaN,NaN
3,The Republican president assumed he was helpin...,http://www.msnbc.com/rachel-maddow-show/auto-i...,msnbc,environment,left,1.0,1.0,Biased,Expresses writer’s opinion,"In Barack Obama’s first term, the administrati...","['rejects', 'happy', 'assumed']",NaN,NaN,NaN
4,The explosion of the Hispanic population has l...,https://www.breitbart.com/politics/2015/02/26/...,breitbart,student-debt,right,1.0,1.0,Biased,No agreement,"Republicans should stop fighting amnesty, Pres...",['explosion'],NaN,NaN,NaN


In [131]:
dfxl_new= df_xl[['topic', 'text', 'biased_words', 'article', 'type']]

In [132]:
dfxl_new.head()

,topic,text,biased_words,article,type
0,elections-2020,YouTube is making clear there will be no “birt...,"['belated', 'birtherism']",YouTube says no ‘deepfakes’ or ‘birther’ video...,center
1,immigration,So while there may be a humanitarian crisis dr...,['crisis'],Speaking to the country for the first time fro...,left
2,abortion,"Looking around the United States, there is nev...","['killing', 'never', 'developing', 'humans', '...",The left has a thing for taking babies hostage...,right
3,environment,The Republican president assumed he was helpin...,"['rejects', 'happy', 'assumed']","In Barack Obama’s first term, the administrati...",left
4,student-debt,The explosion of the Hispanic population has l...,['explosion'],"Republicans should stop fighting amnesty, Pres...",right


In [133]:
print(dfxl_new['type'].value_counts())

type
left      25764
right     25734
center    14608
Name: count, dtype: int64


In [134]:
print(dfxl_new['type'].unique())

['center' 'left' 'right' nan]


In [135]:
dfxl_new.isna().sum()

topic            1762
text                2
biased_words    38512
article         74866
type            11960
dtype: int64

* All json files combined and saved to excel file

In [136]:
# Paths
json_dir = "./llama3_8b"
output_dir = "bias_classifier/llama3_8b"
all_data = []

# Load and preprocess data
for filename in os.listdir(json_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(json_dir, filename)
        with open(file_path, 'r') as f:
            try:
                file_data = json.load(f)
                all_data.extend(file_data)
            except json.JSONDecodeError as e:
                print(f"Error reading {filename}: {e}")

df = pd.DataFrame(all_data)

# --- Normalize 'analysis' field ---
if 'analysis' in df.columns:
    analysis_df = pd.json_normalize(df['analysis'])
    df = pd.concat([df.drop(columns=['analysis']), analysis_df], axis=1)

In [137]:
df.head()

,source,url,title,text,authors,publish_date,keywords,summary,scraped_at,bias_category,reasoning,supporting_examples,additional_analysis,raw_response,model,analyzed_at
0,The Indian Express,https://indianexpress.com/article/technology/t...,You really don’t need a more powerful computer...,You really don’t need a more powerful computer...,"[Nandagopal Rajan, Cdata, Var Template_Content...",2025-04-15 08:04:48+05:30,"[air, macbook, really, m4with, version, need, ...",You really don’t need a more powerful computer...,2025-04-15T17:56:47.989815,Center,,,,BIAS CATEGORY: Center\n\nREASONING:\n\nThe art...,llama3:8b,2025-04-15T22:21:48.742441
1,The Indian Express,https://indianexpress.com/article/india/bihar-...,Even MLCs are not safe: Bihar RJD leader says ...,"Rashtriya Janata Dal’s Mohammad Shoaib, a Memb...","[Himanshu Harsh, Cdata, Var Template_Content, ...",2025-04-15 15:32:09+05:30,"[arrested, mentioned, number, information, sho...","Rashtriya Janata Dal’s Mohammad Shoaib, a Memb...",2025-04-15T17:56:31.289229,Center,The article presents a factual account of an a...,,,BIAS CATEGORY: Center\n\nREASONING: The articl...,llama3:8b,2025-04-15T22:21:51.469832
2,The Indian Express,https://indianexpress.com/article/cities/delhi...,Delhi News Live Updates: Capital braces for he...,Delhi News Live Updates: Visitors at the India...,"[Cdata, Var Template_Content, Sso_Login_Box, X...",2025-04-15 16:37:01+05:30,"[updates, temperature, live, issues, 2025, ass...",Delhi News Live Updates: Visitors at the India...,2025-04-15T17:56:53.078721,Center,The article presents a mix of neutral and fact...,,"Overall, the tone of the article is neutral an...",BIAS CATEGORY: Center\nREASONING: The article ...,llama3:8b,2025-04-15T22:21:55.058438
3,The Indian Express,https://indianexpress.com/article/india/suprem...,‘One has to be careful’: Supreme Court objects...,The Supreme Court Tuesday criticised the Allah...,"[Cdata, Var Template_Content, Sso_Login_Box, X...",2025-04-15 16:04:27+05:30,"[court, trouble, objects, rape, high, careful,...",The Supreme Court Tuesday criticised the Allah...,2025-04-15T17:56:57.528312,Center,The article presents a neutral and objective a...,,The article does not appear to promote a speci...,BIAS CATEGORY: Center\n\nREASONING: The articl...,llama3:8b,2025-04-15T22:22:03.496738
4,The Indian Express,https://indianexpress.com/article/business/ind...,India’s retail inflation slips to over 5-year ...,India’s retail inflation slipped to a more-tha...,"[Cdata, Var Template_Content, Sso_Login_Box, X...",2025-04-15 16:57:01+05:30,"[cuts, prices, slips, low, opens, inflation, r...",India’s retail inflation slipped to a more-tha...,2025-04-15T17:56:58.267002,Center,The article presents a factual report on India...,,,BIAS CATEGORY: Center\n\nREASONING: The articl...,llama3:8b,2025-04-15T22:22:06.094239


In [138]:
df.columns

Index(['source', 'url', 'title', 'text', 'authors', 'publish_date', 'keywords',
       'summary', 'scraped_at', 'bias_category', 'reasoning',
       'supporting_examples', 'additional_analysis', 'raw_response', 'model',
       'analyzed_at'],
      dtype='object')

In [139]:
# Select and rename the relevant columns
df = df.rename(columns={
    'title': 'topic',
    'text': 'text',
    'keywords': 'biased_words',
    'summary': 'article',
    'bias_category': 'type'
})

In [140]:
df_new = df[['topic', 'text', 'biased_words', 'article', 'type']]

In [141]:
df_new.head()

,topic,text,biased_words,article,type
0,You really don’t need a more powerful computer...,You really don’t need a more powerful computer...,"[air, macbook, really, m4with, version, need, ...",You really don’t need a more powerful computer...,Center
1,Even MLCs are not safe: Bihar RJD leader says ...,"Rashtriya Janata Dal’s Mohammad Shoaib, a Memb...","[arrested, mentioned, number, information, sho...","Rashtriya Janata Dal’s Mohammad Shoaib, a Memb...",Center
2,Delhi News Live Updates: Capital braces for he...,Delhi News Live Updates: Visitors at the India...,"[updates, temperature, live, issues, 2025, ass...",Delhi News Live Updates: Visitors at the India...,Center
3,‘One has to be careful’: Supreme Court objects...,The Supreme Court Tuesday criticised the Allah...,"[court, trouble, objects, rape, high, careful,...",The Supreme Court Tuesday criticised the Allah...,Center
4,India’s retail inflation slips to over 5-year ...,India’s retail inflation slipped to a more-tha...,"[cuts, prices, slips, low, opens, inflation, r...",India’s retail inflation slipped to a more-tha...,Center


#### Data Wrangling

### Excel files

* Handling missing values

In [142]:
dfxl_new['text'] = dfxl_new['text'].fillna('')  # Fill with empty string

# Define imputation strategies for each column
imputation_strategies = {
    'topic': 'most_frequent',  # Categorical column
    'biased_words': 'constant',  # Will specify fill_value
    'article': 'constant',      # Will specify fill_value
    'type': 'most_frequent'    # Categorical column
}

# Apply imputation to each column
for col, strategy in imputation_strategies.items():
    if strategy == 'constant':
        # For list-like/text columns, fill with empty string representation
        imputer = SimpleImputer(strategy=strategy, fill_value='[]' if col == 'biased_words' else '')
    else:
        imputer = SimpleImputer(strategy=strategy)
    
    # Reshape to 2D array as required by sklearn
    imputed_values = imputer.fit_transform(dfxl_new[[col]]).ravel()
    dfxl_new[col] = imputed_values

# Verify no missing values remain
print("Missing values after imputation:")


Missing values after imputation:


/var/folders/zl/014zkbv57bb_wpycm8r859740000gn/T/ipykernel_17664/1529717764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfxl_new['text'] = dfxl_new['text'].fillna('')  # Fill with empty string
/var/folders/zl/014zkbv57bb_wpycm8r859740000gn/T/ipykernel_17664/1529717764.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfxl_new[col] = imputed_values
/var/folders/zl/014zkbv57bb_wpycm8r859740000gn/T/ipykernel_17664/1529717764.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [143]:
print(dfxl_new.isna().sum())

topic           0
text            0
biased_words    0
article         0
type            0
dtype: int64


#### Json files

* Mapping the data

In [144]:
print(df_new['type'].unique())

['Center' 'Slightly Left' 'Slightly Right' 'Right' 'Extreme Right'
 'Center/Slightly Right' 'Slightly Center' 'Center/Left' 'Center-Left'
 'Center-Right' 'Center/Slightly Left' 'Slightly left']


In [148]:
bias_mapping = {
    # Center
    'Center': 'center',
    'Center/Slightly Right': 'center',
    'Slightly Center': 'center',
    'Center-Left': 'center',
    'Center-Right': 'center',
    'Center/Slightly Left': 'center',
    
    # Left
    'Slightly Left': 'left',
    'Center/Left': 'left',
    'Slightly left': 'left',  # Note: lowercase 'left' corrected
    
    # Right
    'Slightly Right': 'right',
    'Right': 'right',
    'Extreme Right': 'right'
}

In [149]:
df_new['type'] = df_new['type'].map(bias_mapping)

/var/folders/zl/014zkbv57bb_wpycm8r859740000gn/T/ipykernel_17664/4232125881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['type'] = df_new['type'].map(bias_mapping)


In [150]:
print(df_new['type'].unique())

['center' 'left' 'right']


In [151]:
print(df_new['type'].value_counts())

type
center    3497
right      366
left       333
Name: count, dtype: int64


In [152]:
df_new.isna().sum()

topic           0
text            0
biased_words    0
article         0
type            0
dtype: int64

### Concat both the dataframe


In [153]:
combined_df = pd.concat([dfxl_new, df_new], ignore_index=True)

# Drop ALL rows with ANY NaN values
combined_df_cleaned = combined_df.dropna()

# Save to Excel
combined_df_cleaned.to_excel("./final_data/data.xlsx", index=False)

print(f"Original shape: {combined_df.shape}")
print(f"After dropping NaN: {combined_df_cleaned.shape}")
print("Saved to data.xlsx")

Original shape: (82262, 5)
After dropping NaN: (82262, 5)
Saved to data.xlsx


In [154]:
df_fl = pd.read_excel("./final_data/data.xlsx", engine='openpyxl')

In [155]:
df_fl.head()

,topic,text,biased_words,article,type
0,elections-2020,YouTube is making clear there will be no “birt...,"['belated', 'birtherism']",YouTube says no ‘deepfakes’ or ‘birther’ video...,center
1,immigration,So while there may be a humanitarian crisis dr...,['crisis'],Speaking to the country for the first time fro...,left
2,abortion,"Looking around the United States, there is nev...","['killing', 'never', 'developing', 'humans', '...",The left has a thing for taking babies hostage...,right
3,environment,The Republican president assumed he was helpin...,"['rejects', 'happy', 'assumed']","In Barack Obama’s first term, the administrati...",left
4,student-debt,The explosion of the Hispanic population has l...,['explosion'],"Republicans should stop fighting amnesty, Pres...",right


In [156]:
print(df_fl['type'].unique())

['center' 'left' 'right']


In [157]:
print(df_fl['type'].value_counts())

type
left      38057
right     26100
center    18105
Name: count, dtype: int64


In [158]:
df_fl.columns

Index(['topic', 'text', 'biased_words', 'article', 'type'], dtype='object')

### The accuracy is very low

In [124]:

# Replace NaN values with empty strings for text columns
text_columns = ['text', 'biased_words', 'article']
df_fl[text_columns] = df_fl[text_columns].fillna('')

# Step 1: Preprocess the features
# Encode categorical column 'topic' using one-hot encoding
topic_encoded = pd.get_dummies(df_fl['topic'], prefix='topic', sparse=True)

# Vectorize text columns ('text', 'biased_words', 'article') using TF-IDF
vectorizers = {}
X_text_sparse = []
for col in text_columns:
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')  # Limit features to avoid high dimensionality
    X_col = vectorizer.fit_transform(df_fl[col])
    vectorizers[col] = vectorizer
    X_text_sparse.append(X_col)

# Combine topic (one-hot encoded) and text features (TF-IDF)
X_sparse = hstack([topic_encoded] + X_text_sparse, format='csr')

# Step 2: Encode the target variable
y = df_fl['type']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 3: Apply SMOTE to balance the classes
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_sparse, y_encoded)

# Step 4: Decode the target variable back to original labels
y_resampled = label_encoder.inverse_transform(y_resampled)

# Step 5: Reconstruct the original columns
# Inverse transform 'topic' (one-hot encoded back to categorical)
topic_columns = topic_encoded.columns
topic_resampled = X_resampled[:, :len(topic_columns)].toarray()
topic_labels = np.array([topic_columns[i].replace('topic_', '') for i in topic_resampled.argmax(axis=1)])

# Inverse transform text columns (approximate from TF-IDF)
df_resampled = pd.DataFrame()
for col in text_columns:
    # Get the TF-IDF features for this column
    start_idx = len(topic_columns) + sum(1000 for c in text_columns if c < col)
    end_idx = start_idx + 1000
    X_col_resampled = X_resampled[:, start_idx:end_idx]
    # Use the vectorizer to approximate the text (this is a simplification)
    df_resampled[col] = [' '.join(vectorizers[col].get_feature_names_out()[X_col_resampled[i].nonzero()[1]]) 
                         if len(X_col_resampled[i].nonzero()[1]) > 0 else '' 
                         for i in range(X_resampled.shape[0])]

# Add 'topic' and 'type' to the DataFrame
df_resampled['topic'] = topic_labels
df_resampled['type'] = y_resampled

# Reorder columns to match the original
df_resampled = df_resampled[['topic', 'text', 'biased_words', 'article', 'type']]

# Step 6: Verify the class distribution
print(df_resampled['type'].value_counts())

# Step 7: Save the balanced dataset
df_resampled.to_csv('./final_data/balanced_data.csv', index=False)
print("Balanced dataset saved as 'balanced_data.csv'")


/Users/pranavi/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


type
center    38057
left      38057
right     38057
Name: count, dtype: int64
Balanced dataset saved as 'balanced_data.xlsx'


### Change the code for balancing

In [160]:
import numpy as np
import pandas as pd
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import gc

In [161]:
# Replace NaN values only in text columns
text_columns = ['text', 'biased_words', 'article']
for col in text_columns:
    if df_fl[col].isna().any():
        df_fl[col] = df_fl[col].fillna('')

print("Original class distribution:")
print(df_fl['type'].value_counts())

# Method 1: Balanced dataset with text preservation
# This approach balances classes while preserving text quality

# Step 1: Determine target count for each class
# Option A: Balance to median class size
class_counts = df_fl['type'].value_counts()
median_count = class_counts.median()
target_count = int(median_count)

# Option B: Balance to a specific size (uncomment to use)
# target_count = 500  # Set your desired count here

print(f"Balancing all classes to approximately {target_count} samples")

# Step 2: Process each class separately to create a balanced dataset
balanced_dfs = []

for class_name in df_fl['type'].unique():
    class_df = df_fl[df_fl['type'] == class_name].copy()
    class_size = len(class_df)
    
    if class_size > target_count:
        # Undersample this class
        print(f"Undersampling class '{class_name}' from {class_size} to {target_count}")
        # Randomly sample without replacement
        sampled_df = class_df.sample(n=target_count, random_state=42)
        balanced_dfs.append(sampled_df)
    
    elif class_size < target_count:
        # Oversample this class
        print(f"Oversampling class '{class_name}' from {class_size} to {target_count}")
        
        # Calculate how many samples to add
        samples_needed = target_count - class_size
        
        # Randomly sample with replacement
        # This preserves the text integrity better than synthetic samples
        additional_samples = class_df.sample(n=samples_needed, replace=True, random_state=42)
        
        # Add small random noise to any numeric columns to avoid exact duplicates
        # Skip this step for text columns to preserve text integrity
        numeric_columns = class_df.select_dtypes(include=[np.number]).columns.tolist()
        
        if numeric_columns:
            for col in numeric_columns:
                # Add very small random noise
                noise_scale = class_df[col].std() * 0.01  # 1% of standard deviation
                if noise_scale > 0:
                    additional_samples[col] = additional_samples[col] + np.random.normal(0, noise_scale, len(additional_samples))
        
        # Combine original and additional samples
        combined_df = pd.concat([class_df, additional_samples], ignore_index=True)
        balanced_dfs.append(combined_df)
    
    else:
        # This class already has the target count
        print(f"Class '{class_name}' already has {class_size} samples (target: {target_count})")
        balanced_dfs.append(class_df)

# Step 3: Combine all balanced classes
df_balanced = pd.concat(balanced_dfs, ignore_index=True)

# Verify final class distribution
print("\nFinal class distribution:")
print(df_balanced['type'].value_counts())

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the complete balanced dataset
df_balanced.to_csv('./final_data/complete_balanced_data.csv', index=False)
print("Complete balanced dataset saved as 'complete_balanced_data.csv'")

# Method 2: Alternative approach with more sophisticated SMOTE for text
# This is a more advanced method if you prefer SMOTE-based balancing

def create_advanced_balanced_dataset():
    print("\nAlternative Method: Advanced SMOTE with text preservation")
    
    # Step 1: Separate features and target
    X = df_fl.drop('type', axis=1)
    y = df_fl['type']
    
    # Step 2: Extract non-text features for SMOTE
    # For text datasets, we'll use categorical columns and any numeric features
    categorical_cols = ['topic']  # Add any other categorical columns here
    X_cat = pd.get_dummies(X[categorical_cols], sparse=False)
    
    # Step 3: Define sampling strategy to balance classes
    class_counts = Counter(y)
    sampling_strategy = {cls: target_count for cls in class_counts.keys()}
    
    # Step 4: Apply SMOTE on categorical features only
    smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
    X_cat_resampled, y_resampled = smote.fit_resample(X_cat, y)
    
    # Step 5: Reconstruct full dataset with text
    # Get indices of original samples used in resampling
    original_indices = smote.sample_indices_
    
    # Create a dataframe for resampled data
    df_resampled = pd.DataFrame(columns=df_fl.columns)
    
    # Add original samples
    original_samples = df_fl.iloc[original_indices].copy()
    df_resampled = pd.concat([df_resampled, original_samples], ignore_index=True)
    
    # Get synthetic sample indices
    synthetic_indices = np.setdiff1d(np.arange(len(X_cat_resampled)), smote.sample_indices_)
    
    # For each synthetic sample, find nearest neighbor of same class and copy text
    print(f"Generating {len(synthetic_indices)} synthetic samples with preserved text...")
    
    # Process in batches to avoid memory issues
    batch_size = 1000
    for batch_start in range(0, len(synthetic_indices), batch_size):
        batch_end = min(batch_start + batch_size, len(synthetic_indices))
        batch_indices = synthetic_indices[batch_start:batch_end]
        
        synthetic_samples = []
        for idx in batch_indices:
            # Get class of this synthetic sample
            cls = y_resampled[idx]
            
            # Get one-hot encoded features
            synthetic_features = X_cat_resampled[idx]
            
            # Find original samples of same class
            cls_samples = df_fl[df_fl['type'] == cls]
            cls_indices = cls_samples.index.tolist()
            
            if cls_indices:
                # Get one-hot encoded features of original samples
                cls_X_cat = pd.get_dummies(cls_samples[categorical_cols], sparse=False)
                
                # Find nearest neighbor
                try:
                    # Calculate distances
                    distances = np.sum((cls_X_cat.values - synthetic_features) ** 2, axis=1)
                    nearest_idx = cls_indices[np.argmin(distances)]
                    
                    # Get the nearest sample
                    nearest_sample = df_fl.loc[nearest_idx].copy()
                    
                    # Add small variations to avoid exact duplicates
                    # (but preserve text columns)
                    for col in nearest_sample.index:
                        if col not in text_columns and col != 'type' and isinstance(nearest_sample[col], (int, float)):
                            # Add small random noise to numeric non-text columns
                            col_std = df_fl[col].std()
                            if col_std > 0:
                                nearest_sample[col] += np.random.normal(0, col_std * 0.05)
                    
                    synthetic_samples.append(nearest_sample)
                except:
                    # Fallback: random sample from same class
                    random_idx = np.random.choice(cls_indices)
                    synthetic_samples.append(df_fl.loc[random_idx].copy())
            else:
                # Shouldn't happen if all classes have samples
                print(f"Warning: No samples found for class {cls}")
        
        # Create dataframe batch and append
        if synthetic_samples:
            batch_df = pd.DataFrame(synthetic_samples)
            df_resampled = pd.concat([df_resampled, batch_df], ignore_index=True)
        
        # Clean memory
        gc.collect()
    
    # Verify final class distribution
    print("\nFinal class distribution (advanced method):")
    print(df_resampled['type'].value_counts())
    
    # Shuffle the dataset
    df_resampled = df_resampled.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Save the advanced balanced dataset
    df_resampled.to_csv('./final_data/balanced_data.csv', index=False)
    print("Advanced balanced dataset saved as 'balanced_data.csv'")



Original class distribution:
type
left      38057
right     26100
center    18105
Name: count, dtype: int64
Balancing all classes to approximately 26100 samples
Oversampling class 'center' from 18105 to 26100
Undersampling class 'left' from 38057 to 26100
Class 'right' already has 26100 samples (target: 26100)

Final class distribution:
type
center    26100
left      26100
right     26100
Name: count, dtype: int64
Complete balanced dataset saved as 'complete_balanced_data.csv'


In [162]:
df_resampled.isna().sum()

topic           0
text            0
biased_words    0
article         0
type            0
dtype: int64

### Zip file

In [126]:
import zipfile

# Define paths to your files
files_to_zip = [
    "./train_bert_with_smote_data.py",
    "./final_data/balanced_data.xlsx"
]

# Output zip file path
zip_path = "./final_data/bert_data.zip"

# Create the zip file
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in files_to_zip:
        zipf.write(file, arcname=file.split("/")[-1])

print(f"Zipped files to {zip_path}")


Zipped files to ./final_data/bert_data.zip
